In [1]:
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb

from sklearn import cluster
from sklearn import model_selection
from sklearn import neural_network
from sklearn import preprocessing

/home/user/opt/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.options.display.width = 200
pd.options.display.max_columns = 50

In [3]:
train = pd.read_csv('../data/in/train.csv', delimiter=';')
test = pd.read_csv('../data/in/test.csv', delimiter=';')
#y = train.cardio
#train.drop('cardio', axis=1, inplace=True)
test['cardio'] = -1

In [4]:
train.cardio.value_counts()

0    35021
1    34979
Name: cardio, dtype: int64

In [5]:
train.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
train.columns

Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'], dtype='object')

In [7]:
test.columns

Index(['id', 'age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'smoke', 'alco', 'active', 'cardio'], dtype='object')

In [8]:
test.smoke = test.smoke.replace('None', -1).astype('int32')
test.active = test.active.replace('None', -1).astype('int32')
test.alco = test.alco.replace('None', -1).astype('int32')

In [9]:
test.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.0
mean,50062.686900,19455.528400,1.353733,164.322267,74.120615,130.271300,95.689000,1.368033,1.222700,-0.022033,-0.052067,0.631533,-1.0
std,28906.167148,2477.840922,0.478136,8.213740,14.341540,212.501152,163.852598,0.679166,0.566494,0.423659,0.383789,0.652569,0.0
min,5.000000,8865.000000,1.000000,50.000000,10.000000,-130.000000,-90.000000,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.0
25%,24983.250000,17658.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-1.0
50%,49995.500000,19694.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,-1.0
75%,75232.250000,21320.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,-1.0
max,99997.000000,23705.000000,2.000000,198.000000,183.000000,16020.000000,9100.000000,3.000000,3.000000,1.000000,1.000000,1.000000,-1.0


In [10]:
df_all = pd.concat([train, test])
df_all.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.00000
mean,49999.500000,19464.864590,1.350820,164.348140,74.180167,129.253490,96.347990,1.367220,1.225330,0.05508,0.022020,0.752070,0.04979
std,28867.657797,2470.428376,0.477229,8.211187,14.379494,173.639687,181.437372,0.679922,0.570543,0.33563,0.286629,0.494371,0.80456
min,0.000000,8865.000000,1.000000,50.000000,10.000000,-150.000000,-90.000000,1.000000,1.000000,-1.00000,-1.000000,-1.000000,-1.00000
25%,24999.750000,17662.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000,-1.00000
50%,49999.500000,19700.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.00000,0.000000,1.000000,0.00000
75%,74999.250000,21324.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.00000,0.000000,1.000000,1.00000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.00000,1.000000,1.000000,1.00000


In [11]:
df2 = pd.DataFrame(index=df_all.index)
df2['y'] = df_all.cardio
for c in ['active', 'alco', 'smoke', 'age']:
    df2[c] = df_all[c]
df2.describe()

,y,active,alco,smoke,age
count,100000.00000,100000.000000,100000.000000,100000.00000,100000.000000
mean,0.04979,0.752070,0.022020,0.05508,19464.864590
std,0.80456,0.494371,0.286629,0.33563,2470.428376
min,-1.00000,-1.000000,-1.000000,-1.00000,8865.000000
25%,-1.00000,1.000000,0.000000,0.00000,17662.000000
50%,0.00000,1.000000,0.000000,0.00000,19700.000000
75%,1.00000,1.000000,0.000000,0.00000,21324.000000
max,1.00000,1.000000,1.000000,1.00000,23713.000000


In [12]:
dfs = [df2]
for col in ['ap_hi', 'ap_lo', 'height', 'weight']:
    df = pd.DataFrame()
    df['x'] = df_all[col]
    idx = (np.abs(df.x) > 10).values
    df.x[idx] = df.x[idx] / 10
    df.x = df.x.astype('int32')
    dfs.append(pd.get_dummies(df.x, prefix=col))
df3 = pd.concat(dfs)
del dfs
df3.describe()

,active,age,alco,ap_hi_-1,ap_hi_-10,ap_hi_-11,ap_hi_-12,ap_hi_-13,ap_hi_-14,ap_hi_-15,ap_hi_1,ap_hi_10,ap_hi_11,ap_hi_1102,ap_hi_111,ap_hi_113,ap_hi_1150,ap_hi_12,ap_hi_120,ap_hi_1200,ap_hi_1208,ap_hi_13,ap_hi_130,ap_hi_1301,ap_hi_14,...,height_6,height_7,height_8,height_9,smoke,weight_1,weight_10,weight_11,weight_12,weight_13,weight_14,weight_15,weight_16,weight_17,weight_18,weight_2,weight_20,weight_3,weight_4,weight_5,weight_6,weight_7,weight_8,weight_9,y
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.0000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000
mean,0.752070,19464.864590,0.022020,0.000010,0.000020,0.000010,0.000020,0.000010,0.000010,0.000010,0.002430,0.038930,0.127970,0.000010,0.000010,0.000010,0.000010,0.404080,0.000020,0.000010,0.000010,0.132170,0.000020,0.000020,0.140750,...,0.000150,0.0001,0.000030,0.000050,0.05508,0.000030,0.033960,0.012940,0.004870,0.001940,0.000590,0.000440,0.000310,0.000160,0.000080,0.000060,0.000020,0.000670,0.013200,0.101960,0.297830,0.277600,0.170430,0.082910,0.04979
std,0.494371,2470.428376,0.286629,0.003162,0.004472,0.003162,0.004472,0.003162,0.003162,0.003162,0.049235,0.193429,0.334058,0.003162,0.003162,0.003162,0.003162,0.490716,0.004472,0.003162,0.003162,0.338677,0.004472,0.004472,0.347765,...,0.012247,0.0100,0.005477,0.007071,0.33563,0.005477,0.181127,0.113016,0.069616,0.044003,0.024283,0.020972,0.017604,0.012648,0.008944,0.007746,0.004472,0.025876,0.114131,0.302597,0.457307,0.447817,0.376012,0.275748,0.80456
min,-1.000000,8865.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,-1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00000
25%,1.000000,17662.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.00000
50%,1.000000,19700.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,1.000000,21324.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.00000
max,1.000000,23713.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000

In [13]:
xbg_params = dict(
    learning_rate = 0.05,
    objective = 'binary:logistic',
    eval_metric = 'logloss',
    seed = 1,
    silent = 1,
    max_depth = 5
                )
idx = (df_all.cardio > -1).values
dtrain = xgb.DMatrix(df_all.ix[idx].drop('cardio', axis=1).values, df_all.cardio.values)
xgb.cv(xbg_params, dtrain, nfold=10, num_boost_round=10000, early_stopping_rounds=100, seed=42)

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.680035,0.000198,0.679941,0.000050
1,0.668193,0.000397,0.668003,0.000094
2,0.657458,0.000568,0.657189,0.000134
3,0.647725,0.000759,0.647370,0.000170
4,0.638882,0.000926,0.638427,0.000204
5,0.630803,0.001104,0.630261,0.000230
6,0.623466,0.001261,0.622802,0.000264
7,0.616741,0.001435,0.615982,0.000294
8,0.610611,0.001574,0.609756,0.000331
9,0.605000,0.001734,0.604050,0.000343


In [14]:
xbg_params = dict(
    learning_rate = 0.05,
    objective = 'binary:logistic',
    eval_metric = 'logloss',
    seed = 1,
    silent = 1,
    max_depth = 7,
    alpha = 0.1,
    gamma = 1
                )
idx = (df3.y > -1).values
dtrain = xgb.DMatrix(df3.ix[idx].drop('y', axis=1).values, df3.y.values)
xgb.cv(xbg_params, dtrain, nfold=10, num_boost_round=10000, early_stopping_rounds=100, seed=42)

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.690174,0.000141,0.689860,0.000028
1,0.687511,0.000280,0.686886,0.000059
2,0.685109,0.000414,0.684192,0.000092
3,0.682940,0.000532,0.681739,0.000119
4,0.680987,0.000659,0.679512,0.000143
5,0.679216,0.000775,0.677475,0.000168
6,0.677607,0.000865,0.675615,0.000190
7,0.676175,0.000971,0.673920,0.000208
8,0.674867,0.001052,0.672373,0.000222
9,0.673688,0.001145,0.670956,0.000243
